In [1]:
#modules
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests

### Teams Data
This will ultimately be the main focus dataset. We want to review team seasonal averages, but we have player data to see how positions affect different team metrics. We extract this for the last ten seasons (2010-2019) from the official [NBA website](https://stats.nba.com/teams/traditional/?Season=2019-20&SeasonType=Regular%20Season). Unlike the other data from Sports Reference, it was easier to scrape the NBA website because the SR data for season average is stored as HTML comments. It is a fun extraction challenge, but for another day.

In [6]:
seas = '2011-12'
team_URL = 'https://stats.nba.com/teams/traditional/?Season=' +seas+ '&SeasonType=Regular%20Season'
#webdriver and go to site and get dynamic table
dr = webdriver.Chrome(executable_path=r'C:/bin/chromedriver.exe')
dr.get(team_URL)
table = dr.find_element_by_xpath('//div[@class="nba-stat-table"]//div[1]//table').get_attribute('outerHTML')
dr.close()
#read table
raw_table = pd.read_html(table)[0]
print(raw_table.columns.values)
print()
raw_table.head()

['Unnamed: 0' 'TEAM' 'GP' 'W' 'L' 'WIN%' 'MIN' 'PTS' 'FGM' 'FGA' 'FG%'
 '3PM' '3PA' '3P%' 'FTM' 'FTA' 'FT%' 'OREB' 'DREB' 'REB' 'AST' 'TOV' 'STL'
 'BLK' 'BLKA' 'PF' 'PFD' '+/-' 'GP RANK' 'W RANK' 'L RANK' 'WIN% RANK'
 'MIN RANK' 'PTS RANK' 'FGM RANK' 'FGA RANK' 'FG% RANK' '3PM RANK'
 '3PA RANK' '3P% RANK' 'FTM RANK' 'FTA RANK' 'FT% RANK' 'OREB RANK'
 'DREB RANK' 'REB RANK' 'AST RANK' 'TOV RANK' 'STL RANK' 'BLK RANK'
 'BLKA RANK' 'PF RANK' 'PFD RANK' '+/- RANK']



,Unnamed: 0,TEAM,GP,W,L,WIN%,MIN,PTS,FGM,FGA,...,DREB RANK,REB RANK,AST RANK,TOV RANK,STL RANK,BLK RANK,BLKA RANK,PF RANK,PFD RANK,+/- RANK
0,1,Chicago Bulls,66,50,16,0.758,48.3,96.3,37.4,82.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,San Antonio Spurs,66,50,16,0.758,48.3,103.7,39.6,82.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Oklahoma City Thunder,66,47,19,0.712,48.5,103.1,37.3,79.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Miami Heat,66,46,20,0.697,48.6,98.5,37.1,79.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Indiana Pacers,66,42,24,0.636,48.5,97.7,35.7,81.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
seas = '2011-12'
team_URL = 'https://stats.nba.com/teams/opponent/?Season=' +seas+ '&SeasonType=Regular%20Season'
#webdriver and go to site and get dynamic table
dr = webdriver.Chrome(executable_path=r'C:/bin/chromedriver.exe')
dr.get(team_URL)
table = dr.find_element_by_xpath('//div[@class="nba-stat-table"]//div[1]//table').get_attribute('outerHTML')
dr.close()
#read table
opp_raw_table = pd.read_html(table)[0]
print(opp_raw_table.columns.values)
print()
opp_raw_table.head()

['Unnamed: 0' 'TEAM' 'GP' 'W' 'L' 'MIN' 'OppFGM' 'OppFGA' 'OppFG%'
 'Opp3PM' 'Opp3PA' 'Opp3P%' 'OppFTM' 'OppFTA' 'OppFT%' 'OppOREB' 'OppDREB'
 'OppREB' 'OppAST' 'OppTOV' 'OppSTL' 'OppBLK' 'OppBLKA' 'OppPF' 'OppPFD'
 'OppPTS' '+/-' 'GP RANK' 'W RANK' 'L RANK' 'MIN RANK' 'OppFGM RANK'
 'OppFGA RANK' 'OppFG% RANK' 'Opp3PM RANK' 'Opp3PA RANK' 'Opp3P% RANK'
 'OppFTM RANK' 'OppFTA RANK' 'OppFT% RANK' 'OppOREB RANK' 'OppDREB RANK'
 'OppREB RANK' 'OppAST RANK' 'OppTOV RANK' 'OppSTL RANK' 'OppBLK RANK'
 'OppBLKA RANK' 'OppPF RANK' 'OppPFD RANK' 'OppPTS RANK' '+/- RANK']



,Unnamed: 0,TEAM,GP,W,L,MIN,OppFGM,OppFGA,OppFG%,Opp3PM,...,OppREB RANK,OppAST RANK,OppTOV RANK,OppSTL RANK,OppBLK RANK,OppBLKA RANK,OppPF RANK,OppPFD RANK,OppPTS RANK,+/- RANK
0,1,Chicago Bulls,66,50,16,48.3,34.6,82.1,42.1,4.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,San Antonio Spurs,66,50,16,48.3,38.1,84.2,45.2,6.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Oklahoma City Thunder,66,47,19,48.5,36.4,85.3,42.7,6.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Miami Heat,66,46,20,48.6,34.6,79.8,43.4,7.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Indiana Pacers,66,42,24,48.5,34.9,80.4,43.5,6.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We only need the data columns because the rank columns are dynamic and if we ever need to determine this, we would be able to do so by sorting specific columns

In [12]:
raw_data.columns
data = raw_data[['TEAM', 'GP', 'W', 'L', 'WIN%', 'MIN', 'PTS', 'FGM',
       'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB',
       'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', '+/-']]
opp_data = opp_raw_table[['TEAM', 'OppFGM', 'OppFGA', 'OppFG%', 'Opp3PM', 'Opp3PA', 'Opp3P%',                  'OppFTM', 'OppFTA', 'OppFT%', 'OppOREB', 'OppDREB', 'OppREB', 'OppAST', 'OppTOV',                   'OppSTL', 'OppBLK', 'OppBLKA', 'OppPF', 'OppPFD', 'OppPTS']]
data = pd.merge(data, opp_data, on=['TEAM'])
data.head()

,TEAM,GP,W,L,WIN%,MIN,PTS,FGM,FGA,FG%,...,OppDREB,OppREB,OppAST,OppTOV,OppSTL,OppBLK,OppBLKA,OppPF,OppPFD,OppPTS
0,Chicago Bulls,66,50,16,0.758,48.3,96.3,37.4,82.8,45.2,...,28.6,40.0,18.9,13.3,7.4,5.2,5.9,17.7,17.3,88.2
1,San Antonio Spurs,66,50,16,0.758,48.3,103.7,39.6,82.8,47.8,...,30.9,41.2,19.6,13.7,7.4,5.0,4.4,18.8,17.3,96.5
2,Oklahoma City Thunder,66,47,19,0.712,48.5,103.1,37.3,79.2,47.1,...,28.6,41.3,19.9,14.2,8.2,4.9,8.2,20.0,20.5,96.9
3,Miami Heat,66,46,20,0.697,48.6,98.5,37.1,79.0,46.9,...,28.7,39.8,19.7,16.8,8.3,4.2,5.4,20.5,19.4,92.5
4,Indiana Pacers,66,42,24,0.636,48.5,97.7,35.7,81.4,43.8,...,30.3,42.3,19.6,14.9,6.6,6.0,5.4,21.6,21.7,94.4


Putting it all together, we need to:
- Extract information (for team and opponent team season averages)
- Only include columns of interest
- Add in the year for which the season applies
- Append values into a single dataframe
- Add in end of season champion and runner-up indicators

In [17]:
def get_data_from_site(URL):
    #access web driver and access page content
    dr = webdriver.Chrome(executable_path=r'C:/bin/chromedriver.exe')
    dr.get(URL)
    table = dr.find_element_by_xpath\
            ('//div[@class="nba-stat-table"]//div[1]//table').get_attribute('outerHTML')
    raw_table = pd.read_html(table)[0]
    dr.close()
    return raw_table

def get_data(season, team_URL, opponent_URL):
    #get the data metrics for the team and opponent season averages into a dataframe
    #team average stats
    raw_df = get_data_from_site(team_URL)
    data = raw_df[['TEAM', 'GP', 'W', 'L', 'WIN%', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA',                   '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',                     'BLKA', 'PF', 'PFD', '+/-']]
    #opponent average stats
    opp_raw = get_data_from_site(opponent_URL)
    opp_data = opp_raw[['TEAM', 'OppFGM', 'OppFGA', 'OppFG%', 'Opp3PM', 'Opp3PA', 'Opp3P%',                  'OppFTM', 'OppFTA', 'OppFT%', 'OppOREB', 'OppDREB', 'OppREB', 'OppAST', 'OppTOV',                  'OppSTL', 'OppBLK', 'OppBLKA', 'OppPF', 'OppPFD', 'OppPTS']]
    #merge two tables and add the year for the team stats
    data = pd.merge(data, opp_data, on=['TEAM'])
    data['Year'] = int(i)
    return data

In [18]:
df_team = pd.DataFrame()
#get each year's data
for i in range(2010,2020):
    seas = str(i-1)+'-'+str(i)[2:]
    URL = 'https://stats.nba.com/teams/traditional/?Season=' +seas+ '&SeasonType=Regular%20Season'
    o_URL = 'https://stats.nba.com/teams/opponent/?Season='+ seas+ '&SeasonType=Regular%20Season'
    df_team = pd.concat([df_team, get_data(seas, URL, o_URL)], ignore_index=True)

In [19]:
df_team.head()

,TEAM,GP,W,L,WIN%,MIN,PTS,FGM,FGA,FG%,...,OppREB,OppAST,OppTOV,OppSTL,OppBLK,OppBLKA,OppPF,OppPFD,OppPTS,Year
0,Cleveland Cavaliers,82,61,21,0.744,48.2,102.1,37.8,77.9,48.5,...,38.5,20.3,12.7,6.6,4.0,5.2,20.8,19.4,95.6,2010
1,Orlando Magic,82,59,23,0.720,48.1,102.8,36.6,78.0,47.0,...,40.0,18.9,12.8,6.7,3.5,5.6,22.2,19.9,95.3,2010
2,Los Angeles Lakers,82,57,25,0.695,48.4,101.7,38.3,83.8,45.7,...,42.2,21.9,14.1,7.0,4.4,4.9,21.2,19.4,97.0,2010
3,Dallas Mavericks,82,55,27,0.671,48.5,102.0,38.3,82.4,46.4,...,42.9,20.2,14.6,6.9,4.1,5.5,20.1,19.1,99.3,2010
4,Phoenix Suns,82,54,28,0.659,48.1,110.2,40.7,82.8,49.2,...,42.3,20.8,13.0,7.9,4.5,5.1,21.8,20.9,105.3,2010


In [20]:
#standardize names
df_team[df_team['TEAM']=='LA Clippers']['TEAM'] = 'Los Angeles Clippers'

In [21]:
df_team.dtypes

TEAM        object
GP           int64
W            int64
L            int64
WIN%       float64
MIN        float64
PTS        float64
FGM        float64
FGA        float64
FG%        float64
3PM        float64
3PA        float64
3P%        float64
FTM        float64
FTA        float64
FT%        float64
OREB       float64
DREB       float64
REB        float64
AST        float64
TOV        float64
STL        float64
BLK        float64
BLKA       float64
PF         float64
PFD        float64
+/-        float64
OppFGM     float64
OppFGA     float64
OppFG%     float64
Opp3PM     float64
Opp3PA     float64
Opp3P%     float64
OppFTM     float64
OppFTA     float64
OppFT%     float64
OppOREB    float64
OppDREB    float64
OppREB     float64
OppAST     float64
OppTOV     float64
OppSTL     float64
OppBLK     float64
OppBLKA    float64
OppPF      float64
OppPFD     float64
OppPTS     float64
Year         int64
dtype: object

Adding champion and runner-up indicators for the seasons. Also note again that 2011-12 was a lockout shortened season. Note that the champions file has abbreviations and team names, while this data has only fully spelled out names. We need to merge the championship status using these columns.

In [22]:
#get champions data
champions = pd.read_csv('champions.csv', index_col='Unnamed: 0')
champions.head()

,Year,Champion,Runner-Up,Finals MVP,Top Scorer,Top Rebr,Top Asst,WS Lead,Points,Rebounds,Assists,Win Shares,Champ-Abbr,Runner-Up-Abbr
0,2019,Toronto Raptors,Golden State Warriors,K. Leonard,K. Leonard (732),D. Green (223),D. Green (187),K. Leonard (4.9),732,223,187,4.9,TOR,GSW
1,2018,Golden State Warriors,Cleveland Cavaliers,K. Durant,L. James (748),D. Green (222),L. James (198),L. James (5.2),748,222,198,5.2,GSW,CLE
2,2017,Golden State Warriors,Cleveland Cavaliers,K. Durant,L. James (591),K. Love (191),L. James (141),L. James (4.3),591,191,141,4.3,GSW,CLE
3,2016,Cleveland Cavaliers,Golden State Warriors,L. James,K. Thompson (582),D. Green (228),R. Westbrook (198),L. James (4.7),582,228,198,4.7,CLE,GSW
4,2015,Golden State Warriors,Cleveland Cavaliers,A. Iguodala,L. James (601),D. Howard (238),L. James (169),S. Curry (3.9),601,238,169,3.9,GSW,CLE


In [23]:
#new columns to check merge for
df_team['Champ'] = 0
df_team['Runner-Up'] = 0

#check team status
for champ, ru, year in zip(champions.Champion.values, champions['Runner-Up'].values, champions.Year.values):
    df_team['Champ'][(df_team['TEAM'] == champ) & (df_team['Year']== year)] = 1
    df_team['Runner-Up'][(df_team['TEAM']== ru) & (df_team['Year']== year)] = 1

In [24]:
#update team so that it is abbreviations and not long names
#we only want the most recent abbreviations so exclude the bottom six which were replaced 
team_names = pd.read_csv('team_names.csv', index_col='Unnamed: 0')[:-6]
team_names = team_names.rename(columns={'Franchise':'TEAM'}) #update name to be able to merge
df_team = pd.merge(df_team, team_names, on='TEAM')
df_team = df_team.rename(columns={'Key':'Tm'})

In [25]:
#checking merge happened correctly by looking at the last few years
check = df_team[(df_team['Champ']==1) | (df_team['Runner-Up']==1)]
check.sort_values(by=['Year', 'Runner-Up'])[10:]

,TEAM,GP,W,L,WIN%,MIN,PTS,FGM,FGA,FG%,...,OppSTL,OppBLK,OppBLKA,OppPF,OppPFD,OppPTS,Year,Champ,Runner-Up,Tm
231,Golden State Warriors,82,67,15,0.817,48.1,110.0,41.6,87.0,47.8,...,8.3,3.6,6.0,18.8,19.9,99.9,2015,1,0,GSW
5,Cleveland Cavaliers,82,53,29,0.646,48.2,103.1,37.7,82.2,45.8,...,7.8,4.5,4.1,20.5,18.4,98.7,2015,0,1,CLE
6,Cleveland Cavaliers,82,57,25,0.695,48.4,104.3,38.7,84.0,46.0,...,7.2,4.4,3.9,20.6,20.3,98.3,2016,1,0,CLE
232,Golden State Warriors,82,73,9,0.890,48.5,114.9,42.5,87.3,48.7,...,8.7,4.1,6.1,19.8,20.7,104.1,2016,0,1,GSW
233,Golden State Warriors,82,67,15,0.817,48.2,115.9,43.1,87.1,49.5,...,8.6,3.8,6.8,19.4,19.3,104.3,2017,1,0,GSW
7,Cleveland Cavaliers,82,51,31,0.622,48.5,110.3,39.9,84.9,47.0,...,7.3,4.3,4.0,20.6,18.1,107.2,2017,0,1,CLE
234,Golden State Warriors,82,58,24,0.707,48.1,113.5,42.8,85.1,50.3,...,8.7,3.7,7.5,18.5,19.6,107.5,2018,1,0,GSW
8,Cleveland Cavaliers,82,50,32,0.610,48.1,110.9,40.4,84.8,47.6,...,7.5,4.1,3.8,20.7,18.6,109.9,2018,0,1,CLE
179,Toronto Raptors,82,58,24,0.707,48.5,114.4,42.2,89.1,47.4,...,7.6,4.5,5.3,20.5,21.0,108.4,2019,1,0,TOR
235,Golden State Warriors,82,57,25,0.695,48.3,117.7,44.0,89.8,49.1,...,7.7,3.6,6.4,19.5,21.4,111.2,2019,0,1,GSW


All champions and the outcomes have been incorporated correctly. This dataset will be saved and used for analysis in EDA and modeling

In [26]:
#save to file
df_team.to_csv('teams.csv')